## Join on string and categorical columns
By the end of this lecture you will be able to:
- join on string columns
- join on categorical columns
- do fast-track joins on string columns using categoricals

I recommended that you do the lectures on "String and categorical dtypes" and "Categoricals and the string cache" in Section 3 before doing this lecture.

In [3]:
import polars as pl
import numpy as np
np.random.seed(0)

## Joins on string dtype

We first create a short array with some integers

In [2]:
integerArray = np.array([3,3,1,2])
integerArray

array([3, 3, 1, 2])

For the left `DataFrame` we convert each of the integers to an `id` string that starts with `"id"`. We keep the integers in the `values` column

In [3]:
dfLeft = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integerArray],
            "values":integerArray
        }
    )
)
dfLeft

shape: (4, 2)
┌─────┬────────┐
│ id  ┆ values │
│ --- ┆ ---    │
│ str ┆ i64    │
╞═════╪════════╡
│ id3 ┆ 3      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      │
└─────┴────────┘

We then create the right `DataFrame` that has metadata about each `id`

In [4]:
dfRight = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(1,4)],
            "metadata":[i for i in range(1,4)]
        }
    )
)
dfRight

shape: (3, 2)
┌─────┬──────────┐
│ id  ┆ metadata │
│ --- ┆ ---      │
│ str ┆ i64      │
╞═════╪══════════╡
│ id1 ┆ 1        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3        │
└─────┴──────────┘

When the `id` column is a string dtype we can join these `DataFrames` in the standard way 

In [5]:
(
    dfLeft.join(dfRight,on="id")
)

shape: (4, 3)
┌─────┬────────┬──────────┐
│ id  ┆ values ┆ metadata │
│ --- ┆ ---    ┆ ---      │
│ str ┆ i64    ┆ i64      │
╞═════╪════════╪══════════╡
│ id3 ┆ 3      ┆ 3        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 3        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2        │
└─────┴────────┴──────────┘

Polars cannot use the fast-track algorithm for joining string columns as the algorithm works on integers.

To use the fast-track algorithm the string column must be cast to categorical dtype

## Joins on categorical dtype
We cast the `id` column to categorical dtype

In [6]:
dfLeft = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integerArray],
            "values":integerArray
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
)
dfLeft

shape: (4, 2)
┌─────┬────────┐
│ id  ┆ values │
│ --- ┆ ---    │
│ cat ┆ i64    │
╞═════╪════════╡
│ id3 ┆ 3      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      │
└─────┴────────┘

And we cast the `id` column to categorical for the right `DataFrame`

In [7]:
dfRight = (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(1,4)],
            "metadata":[i for i in range(1,4)]
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
)
dfRight

shape: (3, 2)
┌─────┬──────────┐
│ id  ┆ metadata │
│ --- ┆ ---      │
│ cat ┆ i64      │
╞═════╪══════════╡
│ id1 ┆ 1        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3        │
└─────┴──────────┘

However, if we try to join them we get an `Exception`

In [8]:
(
    dfLeft.join(dfRight,on="id")
)

ComputeError: Joins/or comparisons on categorical dtypes can only happen if they are created under the same global string cache.Hint: set a global StringCache

We get an `Exception` because we didn't cast to categorical for both `DataFrames` inside a `StringCache`. 

As we are not inside a `StringCache` Polars can't be sure if the left and right `DataFrames` use the same mapping from strings to integers.

We try casting to categorical again inside a `StringCache`

In [9]:
with pl.StringCache():
    dfLeft = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in integerArray],
                "values":integerArray
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
    )
    
    dfRight = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in range(1,4)],
                "metadata":[i for i in range(1,4)]
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
    )

We can now join the `DataFrames` in the standard way

In [10]:
(
    dfLeft.join(dfRight,on="id")
)

shape: (4, 3)
┌─────┬────────┬──────────┐
│ id  ┆ values ┆ metadata │
│ --- ┆ ---    ┆ ---      │
│ cat ┆ i64    ┆ i64      │
╞═════╪════════╪══════════╡
│ id3 ┆ 3      ┆ 3        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 3        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1        │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2        │
└─────┴────────┴──────────┘

We can also do the `join` or any other operations inside the `StringCache` block. 

## Fast-track joins
We can do fast-track joins on **sorted** categorical columns as these are integer columns underneath-the-hood.

**Key point**: the categorical join columns must be sorted based on their `physical` integer representation and not their `lexical` alphanumeric representation.

To illustrate this we create `dfLeft` and `dfRight` each with a `physical` integer column

In [11]:
with pl.StringCache():
    dfLeft = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in integerArray],
                "values":integerArray
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
        .with_columns(
            pl.col("id").to_physical().alias("physical_left")
        )
    )
    dfRight = (
        pl.DataFrame(
            {
                "id":[f"id{i}" for i in range(1,4)],
                "metadata":[i for i in range(1,4)]
            }
        )
        .with_columns(
            pl.col("id").cast(pl.Categorical)
        )
        .with_columns(
            pl.col("id").to_physical().alias("physical_right")
        )
    )

We inspect the new left and right `DataFrames` with the `physical` column

In [12]:
dfLeft

shape: (4, 3)
┌─────┬────────┬───────────────┐
│ id  ┆ values ┆ physical_left │
│ --- ┆ ---    ┆ ---           │
│ cat ┆ i64    ┆ u32           │
╞═════╪════════╪═══════════════╡
│ id3 ┆ 3      ┆ 0             │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 0             │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1             │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2             │
└─────┴────────┴───────────────┘

In [13]:
dfRight

shape: (3, 3)
┌─────┬──────────┬────────────────┐
│ id  ┆ metadata ┆ physical_right │
│ --- ┆ ---      ┆ ---            │
│ cat ┆ i64      ┆ u32            │
╞═════╪══════════╪════════════════╡
│ id1 ┆ 1        ┆ 1              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2        ┆ 2              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3        ┆ 0              │
└─────┴──────────┴────────────────┘

From the `physical` columns we can see that:
- `dfLeft` *looks* unsorted (from the alphabetic values in `id`) but is actually sorted (from the integer values in `physical_left`) while 
- `dfRight` *looks* sorted but is actually unsorted!

If we inspect the `flags` we see that Polars doesn't think either is sorted

In [14]:
print(dfLeft["id"].flags)
print(dfRight["id"].flags)

{'SORTED_ASC': False, 'SORTED_DESC': False}
{'SORTED_ASC': False, 'SORTED_DESC': False}


We can use `set_sorted` to tell Polars that `dfLeft` is sorted.

We need to sort `dfRight` by `id`. Recall that by default when we sort a categorical column we sort by the `physical` integer representation.

We create new `DataFrames` here to avoid confusion if cells in this notebook are executed out-of-order

In [15]:
dfLeftSorted = (
    dfLeft
    .with_columns(
        pl.col("id").set_sorted()
    )
)
dfRightSorted = (
    dfRight
    .sort("id")
)
dfRightSorted

shape: (3, 3)
┌─────┬──────────┬────────────────┐
│ id  ┆ metadata ┆ physical_right │
│ --- ┆ ---      ┆ ---            │
│ cat ┆ i64      ┆ u32            │
╞═════╪══════════╪════════════════╡
│ id3 ┆ 3        ┆ 0              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1        ┆ 1              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2        ┆ 2              │
└─────┴──────────┴────────────────┘

We can now join the sorted `DataFrames` and Polars will use the fast-track algorithm

In [16]:
(
    dfLeftSorted.join(dfRightSorted,on="id")
)

shape: (4, 5)
┌─────┬────────┬───────────────┬──────────┬────────────────┐
│ id  ┆ values ┆ physical_left ┆ metadata ┆ physical_right │
│ --- ┆ ---    ┆ ---           ┆ ---      ┆ ---            │
│ cat ┆ i64    ┆ u32           ┆ i64      ┆ u32            │
╞═════╪════════╪═══════════════╪══════════╪════════════════╡
│ id3 ┆ 3      ┆ 0             ┆ 3        ┆ 0              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 0             ┆ 3        ┆ 0              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1             ┆ 1        ┆ 1              │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2             ┆ 2        ┆ 2              │
└─────┴────────┴───────────────┴──────────┴────────────────┘

## Are fast-track joins worthwhile?
A fast-track join may or may not speed up your overall query - you have to check the performance for your data. Factors that affect the performance include:
- size of the `DataFrames` and
- cardinality of the join column (fast-track is more worthwhile with high cardinality)

## Getting fast-track joins on categoricals right

To get fast-track joins right ensure that the categorical column in both the left and right `DataFrames` are sorted.


In the example above the left `DataFrame` was sorted but the right `DataFrame` was not. This is not true in general.

We can check if the `physical` column is sorted by taking the diff and confirming all differences are `>=0`

In [17]:
dfLeft.with_columns(
    pl.col("physical_left").diff().alias("diff")
)

shape: (4, 4)
┌─────┬────────┬───────────────┬──────┐
│ id  ┆ values ┆ physical_left ┆ diff │
│ --- ┆ ---    ┆ ---           ┆ ---  │
│ cat ┆ i64    ┆ u32           ┆ u32  │
╞═════╪════════╪═══════════════╪══════╡
│ id3 ┆ 3      ┆ 0             ┆ null │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 0             ┆ 0    │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1             ┆ 1    │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2             ┆ 1    │
└─────┴────────┴───────────────┴──────┘

However - as the `physical` column has *unsigned* 32-bit integer dtype it cannot represent negative values. So we must cast the `physical` column to a signed integer dtype before taking the `diff` 

In [18]:
dfLeft.with_columns(
    pl.col("physical_left").cast(pl.Int32).diff().alias("diff")
)

shape: (4, 4)
┌─────┬────────┬───────────────┬──────┐
│ id  ┆ values ┆ physical_left ┆ diff │
│ --- ┆ ---    ┆ ---           ┆ ---  │
│ cat ┆ i64    ┆ u32           ┆ i32  │
╞═════╪════════╪═══════════════╪══════╡
│ id3 ┆ 3      ┆ 0             ┆ null │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id3 ┆ 3      ┆ 0             ┆ 0    │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id1 ┆ 1      ┆ 1             ┆ 1    │
├╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ id2 ┆ 2      ┆ 2             ┆ 1    │
└─────┴────────┴───────────────┴──────┘

We check that the minimum value of the diff for the left `DataFrame` is `>=0`

In [19]:
dfLeft.select(
    pl.col("physical_left").cast(pl.Int32).diff().min().alias("min_diff")
)

shape: (1, 1)
┌──────────┐
│ min_diff │
│ ---      │
│ i32      │
╞══════════╡
│ 0        │
└──────────┘

We check that the minimum value of the diff for the right `DataFrame` is `>=0`

In [20]:
dfRight.select(
    pl.col("physical_right").cast(pl.Int32).diff().min().alias("min_diff")
)

shape: (1, 1)
┌──────────┐
│ min_diff │
│ ---      │
│ i32      │
╞══════════╡
│ -2       │
└──────────┘

We see that right `DataFrame` is not sorted

In [21]:
dfRight.select(
    pl.col("physical_right").diff()
)

shape: (3, 1)
┌────────────────┐
│ physical_right │
│ ---            │
│ u32            │
╞════════════════╡
│ null           │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1              │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4294967294     │
└────────────────┘

So in this case we can call `set_sorted` on the `id` column of the left `DataFrame` but we would have to sort the `id` column of the right `DataFrame` to have both `DataFrames` sorted.

## Exercises
In the exercises you will develop your understanding of:
- joining on categorical columns
- joining on string columns
- doing fast-track joins on categorical columns

## Exercise 1
The CITES and ISO CSV files are here 

In [22]:
citesCSVFile = "../data/cites_extract.csv"
isoCSVFile = "../data/countries_extract.csv"

We want to join the ISO data for importers and exporters.

- create `DataFrames` from the CITES trade data and ISO country data in the following CSVs
- cast the join columns to categorical

Join the ISO data for both importers and exporters

Q: Could Polars do a fast-track join on `Importer` in `dfCites` if `set_sorted` was used?

Hint: Add a `physical` column to `dfCites`

Do a fast-track join with ISO data on the `Importer` and `Exporter` columns (combine and modify your code from the first and second parts of this exercise)

## Exercise 2

We compare the performance of sorted and unsorted joins on strings and categoricals. 

We create the left `DataFrame` with length `N` and random `id` strings in this function

In [4]:
N = 100_000
cardinality = N // 2
def createLeftDataFrame(N:int,cardinality:int):
    """
    Create the left dataframe with columns:
    id - random strings of the form idX where X is between 0 and 0
    values - the integer X value
    physical - the physical integers underlying the categorical id column
    """
    # create the random integer array
    integerArray = np.random.randint(0,cardinality,N)
    return (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in integerArray],
            "values":integerArray
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
    .with_columns(
        pl.col("id").to_physical().alias("physical")
    )
)
dfLeft = createLeftDataFrame(N = N,cardinality=cardinality)
dfLeft.head()

shape: (5, 3)
┌─────────┬────────┬──────────┐
│ id      ┆ values ┆ physical │
│ ---     ┆ ---    ┆ ---      │
│ cat     ┆ i64    ┆ u32      │
╞═════════╪════════╪══════════╡
│ id2732  ┆ 2732   ┆ 0        │
│ id43567 ┆ 43567  ┆ 1        │
│ id42613 ┆ 42613  ┆ 2        │
│ id45891 ┆ 45891  ┆ 3        │
│ id21243 ┆ 21243  ┆ 4        │
└─────────┴────────┴──────────┘

We create the right `DataFrame` with metadata about each `id` in this function

In [5]:
def createRightDataFrame(N:int,cardinality:int):
    """
    Create the right dataframe with columns:
    id - the string ids covering the same range as the left dataframe
    meta - a metadata column that has the integer number from the id
    physical - the physical integers underlying the categorical id column
    """
    return (
    pl.DataFrame(
        {
            "id":[f"id{i}" for i in range(cardinality)],
            "meta":[i for i in range(cardinality)]
        }
    )
    .with_columns(
        pl.col("id").cast(pl.Categorical)
    )
    .with_columns(
        pl.col("id").to_physical().alias("physical")
    )

)
dfRight = createRightDataFrame(N = N,cardinality=cardinality)
dfRight.head(3)

shape: (3, 3)
┌─────┬──────┬──────────┐
│ id  ┆ meta ┆ physical │
│ --- ┆ ---  ┆ ---      │
│ cat ┆ i64  ┆ u32      │
╞═════╪══════╪══════════╡
│ id0 ┆ 0    ┆ 0        │
│ id1 ┆ 1    ┆ 1        │
│ id2 ┆ 2    ┆ 2        │
└─────┴──────┴──────────┘

Create `dfLeft` and `dfRight` inside a `StringCache`

In [ ]:
N = 10_000_000
cardinality = 10
<blank>

Time how long it takes to join on unsorted categorical columns

In [ ]:
%%timeit -n1 -r1 
(
    <blank>
)

Sort the categorical columns in new `DataFrames`

In [ ]:
dfLeftSorted = <blank>
dfRightSorted = <blank>

Time how long it takes to join on sorted categorical columns

Cast the categorical columns to strings in new `DataFrames`

In [ ]:
dfLeftString = (dfLeft.<blank>)
dfRightString = (dfright.<blank>)

Time how long it takes to join on string columns

Do these comparisons again with low cardinality with `cardinality = 10`

## Solutions

## Solution to Exercise 1

We want to join the ISO data for importers and exporters.

In a single query:
- create `DataFrames` from the CITES trade data and ISO country data in the following CSVs
- cast the relevant columns to categorical

In [ ]:
with pl.StringCache():
    dfCITES = (
        pl.read_csv(citesCSVFile)
        .with_columns(
            [
                pl.col("Importer").cast(pl.Categorical),
                pl.col("Exporter").cast(pl.Categorical),
            ]
        )
    )
    dfISO = (
        pl.read_csv(isoCSVFile)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
    )

Join the ISO data for importers and exporters

In [ ]:
with pl.StringCache():
    dfCITES = (
        pl.read_csv(citesCSVFile)
        .with_columns(
            [
                pl.col("Importer").cast(pl.Categorical),
                pl.col("Exporter").cast(pl.Categorical),
            ]
        )
    )
    dfISO = (
        pl.read_csv(isoCSVFile)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
    )
    
(
    dfCITES
        .join(dfISO,left_on="Importer",right_on="alpha-2", how="left")
        .rename({"name":"name_importer","region":"region_importer"})
        .join(dfISO,left_on="Exporter",right_on="alpha-2", how="left")
        .rename({"name":"name_exporter","region":"region_exporter"})
)

Q: Could Polars do a fast-track join with `dfCites` on `Importer` if `set_sorted` was used?

Hint: Add a `physical` column to `dfCites` and take the `diff`

In [ ]:
(
    dfCITES
    .with_columns(pl.col("Importer").to_physical().alias("physical_importer")
                )
    .with_columns(pl.col("physical_importer").cast(pl.Int32).diff().alias("diff"))
)

No, we can see negative values in the `diff`

Do a fast-track join on the `Importer` and `Exporter` columns (copy your code from above)

See:
- the sorting on `dfISO`
- the two sort calls on `dfCites` in the join query

In [ ]:
with pl.StringCache():
    dfCITES = (
        pl.read_csv(citesCSVFile)
        .with_columns(
            [
                pl.col("Importer").cast(pl.Categorical),
                pl.col("Exporter").cast(pl.Categorical),
            ]
        )
    )
    dfISO = (
        pl.read_csv(isoCSVFile)
        .with_columns(
                pl.col("alpha-2").cast(pl.Categorical)
        )
        ### Sorting on dfISO!
        .sort("alpha-2")
    )
(
    dfCITES
        .sort("Importer")
        .join(dfISO,left_on="Importer",right_on="alpha-2", how="left")
        .rename({"name":"name_importer","region":"region_importer"})
        .sort("Exporter")
        .join(dfISO,left_on="Exporter",right_on="alpha-2", how="left")
        .rename({"name":"name_exporter","region":"region_exporter"})
)

## Solution to Exercise 2

Create `dfLeft` and `dfRight` inside a `StringCache`

In [23]:
N = 10_000_000
cardinality = 10
with pl.StringCache():
    dfLeft = createLeftDataFrame(N = N,cardinality=cardinality)
    dfRight = createRightDataFrame(N = N,cardinality=cardinality)

Time how long it takes to join on unsorted categorical columns

In [24]:
%%timeit -n1 -r1
(
    dfLeft.join(dfRight,on="id")
)

237 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Sort the categorical columns

In [25]:
dfLeftSorted = dfLeft.sort("id")
dfRightSorted = dfRight.sort("id")

Time how long it takes to join on sorted categorical columns

In [26]:
%%timeit -n1 -r1
(
    dfLeftSorted.join(dfRightSorted,on="id")
)

196 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Cast the categorical columns to strings and compare how long it takes to join on string columns 

In [10]:
dfLeftString = dfLeft.with_columns(pl.col("id").cast(pl.Utf8))
dfRightString = dfRight.with_columns(pl.col("id").cast(pl.Utf8))

In [11]:
%%timeit -n1 -r1
(
    dfLeftString.join(dfRightString,on="id")
)

357 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


The `id` column has high cardinality because we set the range of `id` values to be `N // 2` in `createLeftDataFrame` and `createRightDataFrame`. 

Do these comparisons again with low cardinality with `cardinality = 10`

Joins are much faster on sorted columns when cardinality is high